# Case 2 - Question 3

In this notebook, we will attempt to create a 60/40 Canadian Equity-Bond Mix as a benchmark. We will compare this portfolio with the returns reported from CPPIB and of the reference portfolio.

## Canadian Equity Bond Mix
- Gathering the data to create this was significantly more difficult than expected.
- For the **equity** portion, we chose to use the TSX60 (large-cap Canadian Firms) index as our representative index
    - Dividends are immediately reinvested on dividend date (at close)
- For the **bond** portion, we will use BlockRock's iShares Core Canadian Universe Bond Index ETF (XBB.TO) to track domestic bond returns
    - Similarly, dividends are reinvested on dividend date at close
 
## Timeframe
- The case notes that CPPIB uses fiscal year format. That is,
    - CPPIB reports its annual performance according to its fiscal calendar (April - March) we will take annual earnings using this time range
    - For example: 2000 returns will be from the time frame April 1999 - March 2000
    - This is supported by the return characteristics around the financial crisis

In [126]:
import pandas as pd
from pandasql import sqldf
import matplotlib.pyplot as plt
import numpy as np
import warnings
warnings.filterwarnings('ignore')

In [127]:
tsx60 = pd.read_sas("tsx60.sas7bdat", encoding = 'ISO-8859-1')

In [128]:
tsx60

,GVKEYX,DATADATE,CONM,TIC,DVPSXM,PRCCM
0,118341,1999-03-31,S&P/TSX 60 Index,CI0029,NaN,384.58
1,118341,1999-04-30,S&P/TSX 60 Index,CI0029,NaN,410.85
2,118341,1999-05-31,S&P/TSX 60 Index,CI0029,NaN,400.07
3,118341,1999-06-30,S&P/TSX 60 Index,CI0029,NaN,411.04
4,118341,1999-07-31,S&P/TSX 60 Index,CI0029,NaN,415.10
...,...,...,...,...,...,...
140,118341,2010-11-30,S&P/TSX 60 Index,CI0029,NaN,741.34
141,118341,2010-12-31,S&P/TSX 60 Index,CI0029,NaN,768.65
142,118341,2011-01-31,S&P/TSX 60 Index,CI0029,1.511825,778.65
143,118341,2011-02-28,S&P/TSX 60 Index,CI0029,1.541175,813.29


In [129]:
portfolio_rets = pd.DataFrame(columns = ["monthid", "equity_ret"])

In [130]:
equity_portfolio = tsx60[["DATADATE", "DVPSXM", "PRCCM"]].copy()

In [131]:
equity_portfolio.columns = ["DATE", "DIVIDEND", "PRC"]
equity_portfolio = equity_portfolio.fillna(0)

In [132]:
num_shares = 1
for row in equity_portfolio.iterrows():
    data = row[1]
    
    dividend = data["DIVIDEND"] * num_shares
    
    shares_repurchased = dividend / data.PRC # Div / Prc = shares repurchased
    num_shares += shares_repurchased
    
    equity_portfolio.loc[row[0], "shares_holding"] = num_shares

In [133]:
counter = 0
for i in range(12, len(equity_portfolio), 12):
    portfolio_rets.loc[counter, "monthid"] = i
    portfolio_rets.loc[counter, "equity_ret"] = (equity_portfolio.loc[i, "shares_holding"] * equity_portfolio.loc[i, "PRC"]) / (equity_portfolio.loc[i - 12, "shares_holding"] * equity_portfolio.loc[i - 12, "PRC"]) - 1
    counter += 1
portfolio_rets


,monthid,equity_ret
0,12,0.47925
1,24,-0.22122
2,36,0.008351
3,48,-0.175403
4,60,0.342734
5,72,0.144315
6,84,0.301257
7,96,0.123492
8,108,0.063716
9,120,-0.304391
